In [1]:
cd /wk171/boyochen/TCRI-Benchmark/

/wk171/boyochen/TCRI-Benchmark


In [2]:
%env CUDA_VISIBLE_DEVICES=-1

env: CUDA_VISIBLE_DEVICES=-1


In [3]:
# fet experiment settings from yaml file
from modules.experiment_helper import parse_experiment_settings

experiment_path = 'experiments/baseline.yml'
experiment_settings = parse_experiment_settings(experiment_path)

In [4]:
# load trained model
from modules.model_constructor import create_model_by_experiment_path

model = create_model_by_experiment_path(experiment_path, version='best-prauc')

In [5]:
# load dataset
from modules.data_handler import get_tensorflow_datasets

datasets = get_tensorflow_datasets(**experiment_settings['data'])

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [6]:
import tensorflow as tf
from modules.data_handler import ascii_array_to_string
import pandas as pd

df = pd.DataFrame(columns=['is_RI', 'pred_RI_prob', 'starting_frame'])
for image_sequences, RI_labels, frame_ID_ascii in datasets['valid']:
    starting_frame = list(map(ascii_array_to_string, frame_ID_ascii.numpy()))
    pred = model(image_sequences).numpy()
    is_RI = RI_labels.numpy()
    batch_size = pred.shape[0]
    for i in range(batch_size):
        df.loc[df.shape[0]] = [is_RI[i, 1], pred[i, 1], starting_frame[i]]

In [15]:
df = df.sort_values(['starting_frame'])
df['pred_RI_prob'] = df['pred_RI_prob'].map('{:,.4f}'.format)

In [17]:
df.head(20)

,is_RI,pred_RI_prob,starting_frame
8032,0.0,0.9518,201501C_2015070715
7323,0.0,0.9338,201501C_2015070718
6449,0.0,0.7937,201501C_2015070721
6490,0.0,0.8314,201501C_2015070800
7390,0.0,0.5297,201501C_2015070803
7263,0.0,0.5106,201501C_2015070806
7187,0.0,0.6780,201501C_2015070809
6833,0.0,0.5611,201501C_2015070812
6607,0.0,0.4445,201501C_2015070815
7010,0.0,0.1245,201501C_2015070818
